In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.multioutput import RegressorChain
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold

In [2]:
path = "D:/LG_Radar"
train = pd.read_csv(path+"/train.csv")
test = pd.read_csv(path+"/test.csv")
submission = pd.read_csv(path+"/sample_submission.csv")

In [3]:
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(14): # ignore 'ID'
        rmse = mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    print(all_nrmse)
    return score

In [4]:
train

,ID,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,...,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TRAIN_00001,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,...,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304
1,TRAIN_00002,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,...,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438
2,TRAIN_00003,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,...,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370
3,TRAIN_00004,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,...,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345
4,TRAIN_00005,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,...,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,TRAIN_39603,66.465,103.320,62.27,1,103.150,66.825,30.20,77.83,298.05,...,29.194,16.582,3.410,-26.486,-26.581,-22.772,24.261,-26.491,-26.584,-26.580
39603,TRAIN_39604,66.465,103.321,62.77,1,102.021,66.825,29.21,102.25,270.67,...,29.859,15.659,3.406,-27.308,-27.203,-24.674,23.427,-27.250,-27.334,-27.325
39604,TRAIN_39605,68.504,103.320,64.67,1,103.144,68.864,29.96,102.61,198.07,...,24.720,16.823,3.215,-26.502,-26.687,-22.577,24.301,-26.388,-26.425,-26.601
39605,TRAIN_39606,66.465,103.320,63.67,1,102.025,67.845,30.30,112.60,275.52,...,26.412,15.757,4.216,-26.760,-26.634,-24.066,23.305,-26.536,-26.751,-26.635


In [5]:
train.columns

Index(['ID', 'X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56', 'Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14'],
      dtype='object')

In [6]:
X = np.array(train[list(train.columns[1:57])])
y = np.array(train[list(train.columns[57:])])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, np.array(y), random_state=42)

# Make base models

각각의 모델의 에러를 가중치로 평균내어 예측한다.

In [31]:
from sklearn.tree import ExtraTreeRegressor

In [43]:
models_list=[]
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)
print(lg_nrmse(y_train, rf_model.predict(X_train)))
print(lg_nrmse(y_test, rf_model.predict(X_test)))

[0.09670271964051644, 0.1345721632517782, 0.13186672359605406, 0.07192073482021387, 0.029781159088212893, 0.03930302386084485, 0.04861211090853905, 0.009036646958175666, 0.008965430882527782, 0.014479332867538401, 0.012568602167582417, 0.009024673411063831, 0.008997954369851504, 0.009004457764971593]
0.7371947900127376
[0.25766138905404445, 0.35886539520721955, 0.3505850821448126, 0.1875901595996728, 0.08019460711328945, 0.11834099431345166, 0.12938286333961457, 0.02375178956958381, 0.023517120082524347, 0.03924488944474016, 0.03335460764072608, 0.023655385261588856, 0.023577889904899667, 0.023706885184489487]
1.974703513928995


In [10]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled, X_test_scaled = scaler.transform(X_train), scaler.transform(X_test) 
svm_model = MultiOutputRegressor(SVR())
svm_model.fit(X_train_scaled, y_train)
print(lg_nrmse(y_test, svm_model.predict(X_test_scaled)))

In [25]:
xgb_model = MultiOutputRegressor(XGBRegressor(max_depth=5, learning_rate=0.1, alpha=4, gamma=1, reg_lambda=7))
xgb_model.fit(X_train, y_train)
print(lg_nrmse(y_train, xgb_model.predict(X_train)))
print(lg_nrmse(y_test, xgb_model.predict(X_test)))

[0.2523191488237303, 0.3471716072995733, 0.34291181589665465, 0.17924742818676148, 0.07528309089874936, 0.0861067120607161, 0.12551482815256845, 0.023022719935640458, 0.022867724240378702, 0.03635485902932705, 0.03195408792574978, 0.022998638186077818, 0.022940338792546144, 0.02299854675715869]
1.878007016436511
[0.2561240472193856, 0.35727586271824735, 0.3479654264452222, 0.18822887493532348, 0.08036300316087812, 0.11843145716290114, 0.12995804292448906, 0.023770709108455572, 0.02353078587312592, 0.03925235171604544, 0.033328623048510524, 0.02364746084163248, 0.023614672274852578, 0.02370478280264867]
1.9696195849666984


# Make k-fold model list

In [68]:
kf = KFold(n_splits=5)
model_list=[]
for train_index, test_index in kf.split(X_train):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train_, X_test_ = X_train[train_index], X_train[test_index]
    y_train_, y_test_ = y_train[train_index], y_train[test_index]
    rf_model = RandomForestRegressor()
    rf_model.fit(X_train_, y_train_)
    print(lg_nrmse(y_train_, rf_model.predict(X_train_)))
    print(lg_nrmse(y_test_, rf_model.predict(X_test_)))
    model_list.append(rf_model)

TRAIN: [ 5941  5942  5943 ... 29702 29703 29704] TEST: [   0    1    2 ... 5938 5939 5940]
[0.0968982588077051, 0.13493379906531625, 0.1322291423198844, 0.06990854383380529, 0.029795495303081235, 0.041120252490417626, 0.04863168862442539, 0.008988719402081468, 0.008904755378445591, 0.014576228855578958, 0.012544719796381451, 0.008951184465179122, 0.008947913153435324, 0.008955143795984903]
0.7378870252610653
[0.25725632925758907, 0.35854823526404667, 0.3520502137909118, 0.19044036114193488, 0.08074997011469774, 0.10544975272401833, 0.13074285626407803, 0.02455845832504542, 0.0243004462215293, 0.038963182781578336, 0.03419552426913025, 0.02446037191112769, 0.024386764211662425, 0.024446302771107223]
1.9705080044249217
TRAIN: [    0     1     2 ... 29702 29703 29704] TEST: [ 5941  5942  5943 ... 11879 11880 11881]
[0.09709111226493157, 0.13484122414472058, 0.13206485303098467, 0.070363904189417, 0.02991219815989195, 0.04303463450028892, 0.04888005895021412, 0.009021830818256938, 0.008933

## SVM 

In [9]:
kf = KFold(n_splits=5)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
model_list=[]
for train_index, test_index in kf.split(X_scaled):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train_, X_test_ = X_scaled[train_index], X_scaled[test_index]
    y_train_, y_test_ = y[train_index], y[test_index]
    svm_model = MultiOutputRegressor(SVR())
    svm_model.fit(X_train_, y_train_)
#     print(lg_nrmse(y_train_, svm_model.predict(X_train_)))
    print(lg_nrmse(y_test_, svm_model.predict(X_test_)))
    model_list.append(svm_model)

TRAIN: [ 7922  7923  7924 ... 39604 39605 39606] TEST: [   0    1    2 ... 7919 7920 7921]
[0.2815756897092332, 0.37063937424303056, 0.36987828250168964, 0.1856965413527071, 0.08577237882852964, 0.18691677366001389, 0.14722979041520667, 0.02727669425390407, 0.027008057302565516, 0.04593303162893563, 0.035488417318156794, 0.027144869897271377, 0.027255197314886483, 0.02713577254239472]
2.175947975961388
TRAIN: [    0     1     2 ... 39604 39605 39606] TEST: [ 7922  7923  7924 ... 15841 15842 15843]
[0.2694430091705296, 0.35866987420717134, 0.35615010044913736, 0.18125981809516695, 0.08296816358174403, 0.15275360792786732, 0.14174317582607596, 0.026020854499405364, 0.02583647057696563, 0.041580958653128955, 0.03457091577568738, 0.025805771884371092, 0.025807472663254213, 0.025921746359970705]
2.0623336604218956
TRAIN: [    0     1     2 ... 39604 39605 39606] TEST: [15844 15845 15846 ... 23762 23763 23764]
[0.25626473679844414, 0.37080209850582807, 0.3615477084808052, 0.19393407463650442

In [22]:
X_test_submission = test[list(test.columns[1:57])]
X_test_submission_scaled = scaler.transform(X_test_submission)

C:\Users\hjhhi\anaconda3\envs\python3.9.0\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [23]:
pred=[]
for i in range(len(model_list)):
    pred.append(model_list[i].predict(X_test_submission_scaled))

In [27]:
joblib.dump(pred, "./svm_pred_test.pkl")

['./svm_pred_test.pkl']

In [26]:
lg_nrmse(y_test, np.sum(pred, axis=0)/5)

ValueError: Found input variables with inconsistent numbers of samples: [9902, 39608]

In [27]:
pred_1 = rf_model.predict(X_test)*0.4
pred_2 = svm_model.predict(X_test_scaled) * 0.2
pred_3 = xgb_model.predict(X_test)*0.4

In [29]:
pred = pred_1 + pred_2 + pred_3

In [28]:
lg_nrmse(y_test, pred)

TypeError: list indices must be integers or slices, not tuple

# Submission

In [22]:
import joblib
svm_model = joblib.load("./svm.pkl")
pred_svm = joblib.load("./pred_svm.pkl")

In [12]:
X_test_submission = test[list(test.columns[1:57])]
X_test_submission_scaled = scaler.transform(X_test_submission)

In [20]:
pred_1 = rf_model.predict(X_test_submission)*0.4

NameError: name 'rf_model' is not defined

In [ ]:
pred_2 = pred_svm * 0.2

In [21]:
pred_3 = xgb_model.predict(X_test_submission)*0.4

NameError: name 'xgb_model' is not defined

In [25]:
submission_data = pred_1 + pred_3

In [28]:
submission[list(submission.columns[1:])] = submission_data

In [31]:
submission

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.409515,1.210701,1.083776,13.933078,31.433623,16.756577,3.122686,-26.144023,-26.197439,-22.219550,24.426465,-26.089073,-26.061156,-26.105292
1,TEST_00002,1.461119,1.187049,1.120219,13.868451,31.126204,16.657150,3.105777,-26.138793,-26.162941,-22.235692,24.417072,-26.103904,-26.091830,-26.132909
2,TEST_00003,1.366390,1.038038,1.003808,14.027303,31.619934,16.673731,3.119544,-25.962618,-25.939933,-22.145309,24.470158,-25.925116,-25.893393,-25.882908
3,TEST_00004,1.425013,1.124561,1.045093,14.876444,32.198437,17.084132,3.110118,-25.639896,-25.654441,-21.805200,24.879766,-25.687802,-25.557868,-25.690587
4,TEST_00005,1.339584,1.039257,0.969167,14.823126,31.544785,17.015359,3.142881,-25.675690,-25.739396,-22.102114,24.768741,-25.578993,-25.634174,-25.639194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,1.294672,0.998447,1.012693,12.803720,31.132021,16.575491,3.206131,-26.549717,-26.525038,-22.732996,24.272350,-26.438003,-26.461607,-26.466054
39604,TEST_39605,1.226949,0.896240,0.933956,14.105540,30.992607,16.591816,3.215134,-26.489779,-26.498661,-22.826320,24.361330,-26.442771,-26.478468,-26.412184
39605,TEST_39606,1.264823,0.910235,0.958471,13.050433,31.079248,16.587271,3.161601,-26.567321,-26.585213,-22.770649,24.162510,-26.500153,-26.524040,-26.507245
39606,TEST_39607,1.215846,0.927153,0.943064,12.815964,30.535386,16.598558,3.203439,-26.573993,-26.576417,-22.888135,24.211090,-26.511960,-26.530781,-26.531343


In [30]:
submission.to_csv("./ensemble_submission.csv", index=None)